In [ ]:
import pandas as pd
import re, pickle

In [ ]:
df = pd.read_csv('sample_data/master.csv')


In [ ]:

df = df.dropna()

In [ ]:
reg = re.compile(r'\w+\d\d')

In [ ]:
#GameID -> Team -> Player
#Posessions defined by when someone shoots 2-3 free throws OR a field goal OR turns the ball over
#Look for "makes 2-pt", "makes 3-pt", "misses 2-pt", "misses 3-pt", "makes free throw _ of 2" and "Turnover by"
home = {'Total': 0}
away = {'Total': 0}
master = {}
errCount = 0
for gid in df['GameID'].unique():
  master[gid] = {}

before = df.iloc[0]['GameID']
keyWords = ['makes 2-pt', 'misses 2-pt', 'makes 3-pt', 'misses 3-pt', 'makes free throw 1 of 2', 'misses free throw 1 of 2', 'Turnover by']

#Check for posession-ending play (FT, FGA, TOV)
def keyCheck(string):
  if 'Team' in string:
    return False 
  for key in keyWords:
    if key in string:
      return True
  return False

def clear():
  global home, away
  home = {'Total': 0}
  away = {'Total': 0}

for row in df.iterrows():
  try:
    if row[1]['GameID'] != before:
      for player in home.keys():
        if player == 'Total':
          continue
        master[before][player] = home[player] / home['Total']
      for player in away.keys():
        if player == 'Total':
          continue
        master[before][player] = away[player] / away['Total']
        
      clear()

    homePlay = row[1]['Home']
    awayPlay = row[1]['Away']
    if keyCheck(homePlay) == False and keyCheck(awayPlay) == False:
      continue
    
    
    
    elif keyCheck(homePlay):
      player = reg.search(homePlay).group(0)
      home['Total'] += 1
      if player not in home.keys():
        home[player] = 1
        
      else:
        home[player] += 1

    elif keyCheck(awayPlay):
      player = reg.search(awayPlay).group(0)
      away['Total'] += 1
      if player not in away.keys():
        away[player] = 1
        
      else:
        away[player] += 1
    before = row[1]['GameID']
  except:
    errCount += 1
  
with open('sample_data/possessions.pickle', 'wb') as pik:
  pickle.dump(master, pik)

ratio = round(errCount / df.shape[0], 4)
print(f'Error Ratio: {str(errCount * 100) + "%"}')

Error Ratio: 5500%


In [ ]:
import os
os.listdir()

['.config', 'sample_data', 'drive']

In [ ]:
ratio

0.0